In [2]:
import re
from itertools import tee
from pathlib import Path
from tqdm.notebook import tqdm
from collections import Counter, defaultdict

In [3]:
input_text = Path("../input_data/14").read_text()

In [4]:
lines = input_text.split("\n")
polymer_template = lines[0]

For fast lookups of insertion instructions I use a dictionary of dictionaries.

In [ ]:
insertion_lookup = defaultdict(dict)
pattern = re.compile(r'(?P<first_element>[A-Z])(?P<second_element>[A-Z]) -> (?P<new_element>[A-Z])')

for match in pattern.finditer(input_text):
    first_element, second_element, new_element = match["first_element"], match["second_element"], match["new_element"]
    insertion_lookup[first_element][second_element]=new_element


Since were dealing with insertions halfway through a string a linked list is a proper data structure. This way we can insert in items without having to shift all items at a later index.

In [5]:
class LinkedList:
    def __init__(self, value):
        self.value = value
        self.next_item = None

    def get_last(self):
        current = self
        while current.next_item:
            current = current.next_item

        return current

    @classmethod
    def from_list(cls, items):
        ll = LinkedList(None)
        for n, item in enumerate(items):
            ll += item
        return ll.next_item

    def __iadd__(self, other):
        final_item = self.get_last()

        final_item.next_item = LinkedList(other)
        return self

    def __iter__(self):
        current = self
        yield current.value
        while current := current.next_item:
            yield current.value

    def length(self):
        total_len = 1
        current = self
        while current := current.next_item:
#             print(f"{current.value=}{total_len=}")
            total_len+=1

        return total_len

    def __str__(self):
        return f"<LinkedList: {[x for x in self]}>"

    def __repr__(self):
        return str(self)

In [6]:
polymer_ll = LinkedList.from_list(list(polymer_template))
polymer_ll

<LinkedList: ['B', 'C', 'H', 'C', 'K', 'F', 'F', 'H', 'S', 'K', 'P', 'B', 'S', 'N', 'V', 'V', 'K', 'V', 'S', 'K']>

In [7]:
def polymerization(polymer_template, steps):
    polymer_ll = LinkedList.from_list(list(polymer_template))
    print(f"Step 0: {polymer_ll.length()}")
    second = polymer_ll  # to simulate the 0th element

    for step in tqdm(range(1, steps + 1)):
        second = polymer_ll
        with tqdm(total=polymer_ll.length() -1 , leave=False) as pbar:
            while (first := second) and (second := second.next_item):
                if (level1:= insertion_lookup.get(first.value, None)) and (new_element := level1.get(second.value, None)):
                    new_item = LinkedList(new_element)
                    new_item.next_item = second
                    first.next_item = new_item
#                     print(f"Update {new_element}")
#                 else:
#                     print("No update")

                pbar.update(1)
#                 for (first_condition, second_condition, new_element) in pair_insertions:
#                     if first.value == first_condition and second.value == second_condition:
#                         first.next_item = LinkedList(new_element)
#                         first.next_item.next_item = second
#                         break

        tqdm.write(f"Step {step}: {polymer_ll.length()}")
    return polymer_ll


In [8]:
%%time
c = Counter(polymerization(polymer_template, 10))
display(c.most_common()[0][1] - c.most_common()[-1][1])
display(c)

Step 0: 20


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

Step 1: 39


  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/9728 [00:00<?, ?it/s]

Step 10: 19457


2797

Counter({'B': 3330,
         'C': 938,
         'F': 1352,
         'S': 1955,
         'K': 3289,
         'O': 2067,
         'V': 2237,
         'P': 717,
         'N': 3039,
         'H': 533})

Wall time: 228 ms


## Part 2
`polymerization(polymer_template, 40)` Memory issues after step 20 needs a different approach
Also it took too long...

As a new strategy approach it as a tree.
For every pair of characters in the template you can only get new characters in between. This means you can calculate the result of all 40 steps of the first two characters, count the result and move on to the next pair of characters.

Similarly you can do this for every next step of the first characters.

Example:

Template: AB

instructions:
AB -> C
AC -> D
CB -> C

```
             AB
           /    \ 
          AC    CB
         / \    / \
        AD DC  CC CB
        -------------
         \ /    \ /
         ADC    CCB
           \    /
            ADCCB
```
We don't need the end result (at the bottom of the tree), we only need a count of the characters.

As soon as you know all characters that form from the node with `AC` you can count the characters in the subtree. For the `AC` node that would be `{A: 1, C: 1, D: 1}`. By doing this recursively you can calculate the resulting count much faster.

For a last performance boost you can cache the result of subtrees with an amount of steps left.


In [31]:
from functools import lru_cache


def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

@lru_cache(maxsize=100000, typed=True)
def process_part(a, b, steps_left):
    if steps_left > 0 and (level1:= insertion_lookup.get(a, None)) and (new_element := level1.get(b, None)):
        return process_part(a, new_element, steps_left-1) + process_part(new_element, b, steps_left-1) - Counter([new_element])
    else:
        # end of this part
        return Counter([a, b])


def polymerization_tree(polymer_template, steps):
    # walk the polymerization as a depth first tree, flatten subtrees which are complete
    global_counter = Counter()
    print(polymer_template)
    for a, b in pairwise(list(polymer_template)):
        # print(f"Processing {a} {b}")
        global_counter += process_part(a, b, steps_left=steps)

    return global_counter - Counter(polymer_template[1:-1])

In [26]:
%%time
c = polymerization_tree(polymer_template, 40)
display(c)
c.most_common()[0][1] - c.most_common()[-1][1]

BCHCKFFHSKPBSNVVKVSK


Counter({'B': 3397499171538,
         'C': 984258259007,
         'F': 1483618435179,
         'S': 1901381229051,
         'K': 3537050036671,
         'O': 2595523651777,
         'V': 2298821733898,
         'P': 778268121366,
         'N': 3304063632119,
         'H': 610236657139})

Wall time: 30 ms


2926813379532

Just for the fun of it, how many steps can this approach handle?

In [30]:
%%time
c = polymerization_tree(polymer_template, 1400)
display(c)
c.most_common()[0][1] - c.most_common()[-1][1]

BCHCKFFHSKPBSNVVKVSK


Counter({'B': 85497600718513471020669328927531476071522658933991767831747779947249992034363873234006765851841469910924184889545437005259973862788707918639664108448330046385339115805077316765995102076403617173874064440078905689789646862985079188890014539580153025285226774421848253517976409656491833310649845182506492463285251752655104615100209260444864252932823173530080796519626551718964002118139483029192689785676335878891095285069155,
         'C': 24768972252314673550507888528971913247656102146129229943923065275725437902987324749590854107233238278969865268441088985615415173553621277027154856582054389233492309270780276339868381612169353278542277060777806576704132988876154227788164467869515163659762634178254879191502765873471333684131594694964649758799311460013243489940816121436818701022443198119544034675906968788327193987369741396014168596472340098376711413855156,
         'F': 37334709449664560219516367404937102454567531625515358729297226731017542315911336942208896737999575682958556844

Wall time: 2.39 s


73652184591979630975921280044827564238518052134254035298404917143145158206854848628876089644920374527937254988242282725553556956521520941294929466889434710959079837869543699393171163522254107154308437120412848167663493079474621687532877659995333076858723676289690922129436120492292709790664545849494373422027649428038731398441437772203095705352246623202066662513711844009304182555537802994792676356782923361796537736127525

It is possible to increase the stack limit, but that would be too much effort :D